In [1]:
from jupyter_set_up import init_django

init_django('pictionary')

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from datetime import datetime

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)

        x = torch.randn(28,28).view(-1,1,28,28)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 50)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = np.prod(x[0].shape)
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [13]:
def train_X_y(start_index, end_index):
    for i, x in enumerate(np.load('train.npy', allow_pickle=True)):
        if start_index<=i<end_index:
            yield i, torch.tensor(int(x[0])), torch.tensor(x[1])

In [5]:
def file_len(file):
    with open('test.npy', 'rb') as f:
        np.lib.format.read_magic(f)
        return np.lib.format.read_array_header_1_0(f)[0][0]

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(cnn, file_len):
    BATCH_SIZE = 10000
    EPOCHS = 1
    optimizer = optim.Adam(cnn.parameters(), lr=0.01)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, file_len, BATCH_SIZE)):
            
            batch_X = []
            batch_y = []
            
            for j, label, img in train_X_y(i, i+BATCH_SIZE):
                batch_y.append(label)
                batch_X.append(img)
                    
            batch_X = torch.stack(batch_X).view(-1, 1, 28, 28).to(device)
            batch_y = torch.stack(batch_y).to(device)

            cnn.zero_grad()
            outputs = cnn(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
        print("Epoch: {} Loss: {}".format(epoch, loss))


def test(cnn, test_X, test_y, file_len):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(file_len)):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = cnn(test_X[i].view(-1, 1, 28, 28).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: {}".format(round(correct / total, 3)))

In [7]:
start_time = datetime.now()
cnn = CNN().to(device)
train(cnn, file_len('train.npy'))
# test(cnn, file_len('test.npy'))
print("Duration: {}".format(datetime.now() - start_time))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [51:57<00:00, 33.17s/it]


Epoch: 0 Loss: 3.5873355865478516
Duration: 0:51:58.850420


In [26]:
y = np.load('test.npy', allow_pickle=True)[:, 0]
y = y.astype('int')
temp = np.load('test.npy', allow_pickle=True)[:, 1]

X = []
for img in temp:
    X.append(torch.tensor(img))

In [24]:
start_time = datetime.now()
test(cnn, X, torch.Tensor(y).to(device), file_len('test.npy'))
print("Duration: {}".format(datetime.now() - start_time))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 933509/933509 [11:09<00:00, 1394.54it/s]

Accuracy: 0.0
Duration: 0:11:09.666488
